Per entrar a una sessió de Spark i iniciar un builder en DeltaLake

In [39]:
#!pip install pyspark
#!pip install delta-spark

import pyspark
from delta import *
from pyspark.sql.functions import col, lit, pi, atan, exp

builder = pyspark.sql.SparkSession.builder.appName("Shops_Deltalake") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

## Comandes com si estessis a una sessió de Spark

Llegir els arxius i guardar-los en format DeltaLake, que ens garantitza que es compleixen les restriccions ACID, i dona avanatatges 




Carregar i llegir els 3 arxius de tenim

In [66]:
#Arxiu Parquet 
shops = spark.read.parquet("./datalake/shops_data/2024-04-17_shops_data.parquet")

shops = shops.drop('attributes.osm_id2')\
                .drop('attributes.abandoned')\
                .drop('attributes.addr_housename')\
                .drop('attributes.addr_housenumber')\
                .drop('attributes.addr_street')\
                .drop('attributes.addr_city')\
                .drop('attributes.addr_state')\
                .drop('attributes.addr_province')\
                .drop('attributes.addr_country')\
                .drop('attributes.addr_district')\
                .drop('attributes.addr_subdistrict')\
                .drop('attributes.addr_unit')\
                .drop('attributes.amenity')\
                .drop('attributes.brand')\
                .drop('attributes.building')\
                .drop('attributes.operator')

shops = shops.withColumnRenamed('attributes.objectid', 'Id')\
            .withColumnRenamed('attributes.addr_postcode', 'Zipcode')\
            .withColumnRenamed('attributes.name', 'Name')\
            .withColumnRenamed('attributes.shop', 'Shop Type')\
            .withColumnRenamed('geometry.x', 'Coordinate X')\
            .withColumnRenamed('geometry.y', 'Coordinate Y')


In [67]:
from uszipcode import SearchEngine
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pandas as pd

# Crea un objeto SearchEngine
search = SearchEngine()

# Función para obtener el código postal
def get_zipcode(lat, lon):
    result = search.by_coordinates(lat=float(lat), lng=float(lon), returns=1)
    return result[0].zipcode if result else None

R = 6378137 #radi de la terra

shops = shops.withColumn('Longitude', (col('Coordinate X') / lit(R)) * (180 / pi()))\
            .withColumn('Latitude',  (((pi() / 2) - 2 * atan(exp(-col('Coordinate Y') / lit(R)))) * (180 / pi())))\
            .drop('Coordinate X')\
            .drop('Coordinate Y')


#Falta arreglar això
shops = shops.withColumn('ZipCode', (get_zipcode('Latitude','Longitude')))
shops.show()

Latitude


ValueError: You can either specify all of `lat`, `lng`, `radius` or none of them

In [6]:
#Guardar l'arxiu
shops.write.mode("overwrite").format("delta").save("./deltalake/shops_data/")

Py4JJavaError: An error occurred while calling o48.save.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at io.delta.storage.HadoopFileSystemLogStore.listFrom(HadoopFileSystemLogStore.java:59)
	at org.apache.spark.sql.delta.storage.LogStoreAdaptor.listFrom(LogStore.scala:452)
	at org.apache.spark.sql.delta.storage.DelegatingLogStore.l

In [7]:


# Aplica la función a tus datos
#df["ZipCode"] = df.apply(lambda row: get_zipcode(row["LATITUDE_X"], row["LONGITUDE_X"]), axis=1)


Download C:\Users\paula\.uszipcode\simple_db.sqlite from https://github.com/MacHu-GWU/uszipcode-project/releases/download/1.0.1.db/simple_db.sqlite ...
  1.00 MB downloaded ...
  2.00 MB downloaded ...
  3.00 MB downloaded ...
  4.00 MB downloaded ...
  5.00 MB downloaded ...
  6.00 MB downloaded ...
  7.00 MB downloaded ...
  8.00 MB downloaded ...
  9.00 MB downloaded ...
  10.00 MB downloaded ...
  11.00 MB downloaded ...
  Complete!


In [21]:
import math

# Coordenadas en Web Mercator
x = -17765273.38688436
y = 2500876.4380595586

R = 6378137 #radi de la terra

# Convertir x a longitud
longitude = (x / R) * (180 / math.pi)

# Convertir y a latitud
latitude = (math.pi / 2 - 2 * math.atan(math.exp(-y / R))) * (180 / math.pi)

#Convertir latitud i longitud a zipcode
get_zipcode(latitude, longitude)

'96741'